![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/14.German_Healthcare_Models.ipynb)

## Colab Setup

In [ ]:
import json

with open('workshop_license_keys_365.json') as f_in:
    license_keys = json.load(f_in)

license_keys.keys()

dict_keys(['PUBLIC_VERSION', 'JSL_VERSION', 'SECRET', 'SPARK_NLP_LICENSE', 'AWS_ACCESS_KEY_ID', 'AWS_SECRET_ACCESS_KEY', 'SPARK_OCR_LICENSE', 'SPARK_OCR_SECRET'])

In [ ]:
import os

# Install java
! apt-get update -qq
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! java -version

secret = license_keys['SECRET']

os.environ['SPARK_NLP_LICENSE'] = license_keys['SPARK_NLP_LICENSE']
os.environ['AWS_ACCESS_KEY_ID']= license_keys['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY'] = license_keys['AWS_SECRET_ACCESS_KEY']
version = license_keys['PUBLIC_VERSION']
jsl_version = license_keys['JSL_VERSION']

! pip install --ignore-installed -q pyspark==2.4.4

! python -m pip install --upgrade spark-nlp-jsl==$jsl_version  --extra-index-url https://pypi.johnsnowlabs.com/$secret

! pip install --ignore-installed -q spark-nlp==$version

import sparknlp

print (sparknlp.version())

import json
import os
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession


from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.base import *
import sparknlp_jsl
from pyspark.sql.types import StructType, StructField, StringType
import itertools

spark = sparknlp_jsl.start(secret)

## Clinical Named Entitiy Recognition (NER)

image.png

In [ ]:

documentAssembler = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")

# Sentence Detector annotator, processes various sentences per line
sentenceDetector = SentenceDetector()\
  .setInputCols(["document"])\
  .setOutputCol("sentence")\

# Tokenizer splits words in a relevant format for NLP
tokenizer = Tokenizer()\
  .setInputCols(["sentence"])\
  .setOutputCol("token")\

word_embeddings = WordEmbeddingsModel.pretrained("w2v_cc_300d",'de','clinical/models')\
  .setInputCols(["sentence", 'token'])\
  .setOutputCol("embeddings")

clinical_ner = NerDLModel.pretrained("ner_healthcare",'de','clinical/models') \
  .setInputCols(["sentence", "token", "embeddings"]) \
  .setOutputCol("ner")

clinical_ner_converter = NerConverterInternal() \
  .setInputCols(["sentence", "token", "ner"]) \
  .setOutputCol("ner_chunk")\

clinical_ner_pipeline = Pipeline(
      stages = [
      documentAssembler,
      sentenceDetector,
      tokenizer,
      word_embeddings,
      clinical_ner,
      clinical_ner_converter
      ])

empty_df = spark.createDataFrame([['']]).toDF("text")

clinical_ner_model = clinical_ner_pipeline.fit(empty_df)

clinical_light_model = LightPipeline(clinical_ner_model)

w2v_cc_300d download started this may take some time.
Approximate size to download 1.2 GB
[OK!]
ner_healthcare download started this may take some time.
Approximate size to download 14.1 MB
[OK!]


In [ ]:
clinical_ner.getClasses()

['O',
 'B-MEDICATION',
 'B-BODY_PART',
 'B-MEDICAL_CONDITION',
 'I-MEDICAL_CONDITION',
 'B-STATE_OF_HEALTH',
 'B-BODY_FLUID',
 'B-BIOLOGICAL_PARAMETER',
 'B-MEASUREMENT',
 'I-MEASUREMENT',
 'B-TREATMENT',
 'B-BIOLOGICAL_CHEMISTRY',
 'B-TIME_INFORMATION',
 'I-TIME_INFORMATION',
 'B-PERSON',
 'B-MEDICAL_SPECIFICATION',
 'I-MEDICATION',
 'B-PROCESS',
 'B-DIAGLAB_PROCEDURE',
 'I-STATE_OF_HEALTH',
 'B-DOSING',
 'I-DOSING',
 'B-MEDICAL_DEVICE',
 'I-PERSON',
 'B-LOCAL_SPECIFICATION',
 'I-BIOLOGICAL_PARAMETER',
 'I-TREATMENT',
 'I-DIAGLAB_PROCEDURE',
 'I-LOCAL_SPECIFICATION',
 'I-MEDICAL_DEVICE',
 'I-BIOLOGICAL_CHEMISTRY',
 'I-BODY_PART',
 'B-TISSUE',
 'I-MEDICAL_SPECIFICATION',
 'I-PROCESS',
 'B-DEGREE']

In [ ]:
import pandas as pd

def get_ner_df (light_model, text):

  light_result = light_model.fullAnnotate(text)

  chunks = []
  entities = []

  for n in light_result[0]['ner_chunk']:
          
      chunks.append(n.result)
      entities.append(n.metadata['entity']) 


  df = pd.DataFrame({'chunks':chunks, 'entities':entities})

  return df


In [ ]:
text='''
Das Kleinzellige Bronchialkarzinom (Kleinzelliger Lungenkrebs, SCLC) ist Hernia femoralis, Akne, einseitig, ein hochmalignes bronchogenes Karzinom, das überwiegend im Zentrum der Lunge, in einem Hauptbronchus entsteht. Die mittlere Prävalenz wird auf 1/20.000 geschätzt. Vom SCLC sind hauptsächlich Peronen mittleren Alters (27-66 Jahre) mit Raucheranamnese betroffen. 
'''

df = get_ner_df (clinical_light_model, text)

df

chunks               entities
0        Kleinzellige            MEASUREMENT
1   Bronchialkarzinom      MEDICAL_CONDITION
2       Kleinzelliger              BODY_PART
3         Lungenkrebs      MEDICAL_CONDITION
4                SCLC      MEDICAL_CONDITION
5              Hernia      MEDICAL_CONDITION
6           femoralis    LOCAL_SPECIFICATION
7                Akne      MEDICAL_CONDITION
8           einseitig            MEASUREMENT
9        hochmalignes      MEDICAL_CONDITION
10       bronchogenes    LOCAL_SPECIFICATION
11           Karzinom              BODY_PART
12              Lunge              BODY_PART
13      Hauptbronchus              BODY_PART
14           mittlere            MEASUREMENT
15          Prävalenz      MEDICAL_CONDITION
16               SCLC      DIAGLAB_PROCEDURE
17        27-66 Jahre            MEASUREMENT
18    Raucheranamnese  MEDICAL_SPECIFICATION
19          betroffen      MEDICAL_CONDITION

## ICD-10GM Pipeline

In [ ]:
# Annotator that transforms a text column from dataframe into an Annotation ready for NLP

documentAssembler = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")

# Sentence Detector annotator, processes various sentences per line

sentenceDetector = SentenceDetector()\
  .setInputCols(["document"])\
  .setOutputCol("sentence")\

# Tokenizer splits words in a relevant format for NLP

tokenizer = Tokenizer()\
  .setInputCols(["sentence"])\
  .setOutputCol("raw_token")\

# Tokenizer splits words in a relevant format for NLP

stopwords = StopWordsCleaner.pretrained('stopwords_de','de')\
  .setInputCols(["raw_token"])\
  .setOutputCol("token")\
  .setCaseSensitive(False)

de_embeddings = WordEmbeddingsModel.pretrained("w2v_cc_300d",'de','clinical/models')\
  .setInputCols(["sentence", 'token'])\
  .setOutputCol("embeddings")\
  .setCaseSensitive(False)

de_ner = NerDLModel.pretrained("ner_healthcare",'de','clinical/models') \
  .setInputCols(["sentence", "token", "embeddings"]) \
  .setOutputCol("ner")

ner_converter = NerConverterInternal() \
  .setInputCols(["sentence", "token", "ner"]) \
  .setOutputCol("ner_chunk")\
  .setWhiteList(['MEDICAL_CONDITION'])

chunk_embeddings = ChunkEmbeddings()\
    .setInputCols("ner_chunk", "embeddings")\
    .setOutputCol("chunk_embeddings")

# ICD resolution model
icd10_resolution = ChunkEntityResolverModel.pretrained("chunkresolve_ICD10GM",'de','clinical/models') \
  .setInputCols(["token", "chunk_embeddings"]) \
  .setOutputCol("icd10_de_code")\
  .setDistanceFunction("EUCLIDEAN") \
  .setNeighbours(5)

pipeline_icd10 = Pipeline(
    stages = [
    documentAssembler,
    sentenceDetector,
    tokenizer,
    stopwords,
    de_embeddings,
    de_ner,
    ner_converter,
    chunk_embeddings,
    icd10_resolution
  ])

empty_df = spark.createDataFrame([['']]).toDF("text")

model_icd10 = pipeline_icd10.fit(empty_df)


stopwords_de download started this may take some time.
Approximate size to download 2.9 KB
[OK!]
w2v_cc_300d download started this may take some time.
Approximate size to download 1.2 GB
[OK!]
ner_healthcare download started this may take some time.
Approximate size to download 14.1 MB
[OK!]
chunkresolve_ICD10GM download started this may take some time.
Approximate size to download 14.8 MB
[OK!]


In [ ]:
de_embeddings.getCaseSensitive()

False

In [ ]:
de_embeddings.getStorageRef()

'w2v_cc_300d'

In [ ]:
de_ner.getStorageRef()

'w2v_cc_300d'

In [ ]:
light_pipeline_icd10 = LightPipeline(model_icd10)


In [ ]:
text='''
Das Kleinzellige Bronchialkarzinom (Kleinzelliger Lungenkrebs, SCLC) ist Hernia femoralis, Akne, einseitig, ein hochmalignes bronchogenes Karzinom, das überwiegend im Zentrum der Lunge, in einem Hauptbronchus entsteht. Die mittlere Prävalenz wird auf 1/20.000 geschätzt. Vom SCLC sind hauptsächlich Peronen mittleren Alters (27-66 Jahre) mit Raucheranamnese betroffen. Etwa 70% der Patienten mit SCLC haben bei Stellung der Diagnose schon extra-thorakale Symptome. Zu den Symptomen gehören Thoraxschmerz, Dyspnoe, Husten und pfeifende Atmung. Die Beteiligung benachbarter Bereiche verursacht Heiserkeit, Dysphagie und Oberes Vena-cava-Syndrom (Obstruktion des Blutflusses durch die Vena cava superior). Zusätzliche Symptome als Folge einer Fernmetastasierung sind ebenfalls möglich. Rauchen und Strahlenexposition sind synergistisch wirkende Risikofaktoren. Die industrielle Exposition mit Bis (Chlormethyläther) ist ein weiterer Risikofaktor. Röntgenaufnahmen des Thorax sind nicht ausreichend empfindlich, um einen SCLC frühzeitig zu erkennen. Röntgenologischen Auffälligkeiten muß weiter nachgegangen werden, meist mit Computertomographie. Die Diagnose wird bioptisch gesichert. Patienten mit SCLC erhalten meist Bestrahlung und/oder Chemotherapie. In Hinblick auf eine Verbesserung der Überlebenschancen der Patienten ist sowohl bei ausgedehnten und bei begrenzten SCLC eine kombinierte Chemotherapie wirksamer als die Behandlung mit Einzelsubstanzen. Es kann auch eine prophylaktische Bestrahlung des Schädels erwogen werden, da innerhalb von 2-3 Jahren nach Behandlungsbeginn ein hohes Risiko für zentralnervöse Metastasen besteht. Das Kleinzellige Bronchialkarzinom ist der aggressivste Lungentumor: Die 5-Jahres-Überlebensrate beträgt 1-5%, der Median des gesamten Überlebens liegt bei etwa 6 bis 10 Monaten.
'''

In [ ]:
import pandas as pd

light_result = light_pipeline_icd10.annotate(text)

pred_df = pd.DataFrame(list(zip(light_result['ner_chunk'], light_result['icd10_de_code'])),
                  columns = ['Problem','ICD10-Code'])

pred_df

Problem ICD10-Code
0           Bronchialkarzinom      I50.0
1                 Lungenkrebs      B90.9
2                        SCLC      C83.0
3                      Hernia        K44
4                        Akne        L70
5                hochmalignes      I37.0
6                    Karzinom     D18.03
7                   Prävalenz      Z83.3
8                   betroffen      R29.6
9                    Symptome      R45.5
10                  Symptomen      F50.5
11              Thoraxschmerz     M99.30
12                    Dyspnoe      R06.0
13                     Husten        R05
14                 Heiserkeit      J38.4
15                  Dysphagie        R13
16                Blutflusses      I66.8
17                       cava      Q26.0
18                   Symptome      R45.5
19             Risikofaktoren      Z83.7
20                 Exposition      W91.9
21           Chlormethyläther      I37.0
22               Risikofaktor        Z57
23  Risiko für zentralnervöse      A48.3
24                 Metastasen      Z80.6
25          Bronchialkarzinom      I50.0
26                Lungentumor     C90.31

In [ ]:
import pandas as pd
idc10_de_data= pd.read_csv('idc10_de_data_0906.csv')

In [ ]:
idc10_de_data[idc10_de_data.code.isin(pred_df['ICD10-Code'])]

code  ...                                   norm_description
260     A48.3  ...  Syndrom toxischen Schocks Sonstige bakterielle...
873     B90.9  ...  Folgezustände Tuberkulose Atmungsorgane näher ...
1381    C83.0  ...    Kleinzelliges BZellLymphom follikuläres Lymphom
1441   C90.31  ...  Solitäres Plasmozytom kompletter Remission Pla...
1705   D18.03  ...  Hämangiom Hepatobiliäres System Pankreas Häman...
3024    F50.5  ...       Erbrechen psychischen Störungen Essstörungen
4434    I37.0  ...  Pulmonalklappenstenose Pulmonalklappenkrankheiten
4520    I50.0  ...            Rechtsherzinsuffizienz Herzinsuffizienz
4602    I66.8  ...  Verschluss Stenose sonstiger zerebraler Arteri...
4950    J38.4  ...  Larynxödem Krankheiten Stimmlippen Kehlkopfes ...
5443      K44  ...        Hernia diaphragmatica Hernia diaphragmatica
6089      L70  ...                                          Akne Akne
10163  M99.30  ...  Knöcherne Stenose Spinalkanals Kopfbereich okz...
11886   Q26.0  ...  Angeborene Stenose cava Angeborene Fehlbildung...
12485     R05  ...                                      Husten Husten
12487   R06.0  ...                           Dyspnoe Störungen Atmung
12518     R13  ...                                Dysphagie Dysphagie
12587   R29.6  ...  Sturzneigung anderenorts klassifiziert Sonstig...
12632   R45.5  ...         Feindseligkeit Symptome Stimmung betreffen
15344   W91.9  ...                Strahlenunfall Exposition Strahlung
15726     Z57  ...  Berufliche Exposition Risikofaktoren Beruflich...
15787   Z80.6  ...  Leukämie Familienanamnese Bösartige Neubildung...
15796   Z83.3  ...  Diabetes mellitus Familienanamnese spezifische...
15800   Z83.7  ...  Krankheiten Verdauungssystems Familienanamnese...

[24 rows x 5 columns]

## Get ICD10 from a phrase given by a user

In [ ]:
def icd_phrase_resolver(distance):

  document = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("document")

  chunk = Doc2Chunk()\
      .setInputCols("document")\
      .setOutputCol("chunk")\

  tokenizer = Tokenizer()\
    .setInputCols(["document"])\
    .setOutputCol("raw_token")\

  stopwords = StopWordsCleaner.pretrained('stopwords_de','de')\
    .setInputCols(["raw_token"])\
    .setOutputCol("token")\
    .setCaseSensitive(False)

  word_embeddings = WordEmbeddingsModel.pretrained("w2v_cc_300d",'de','clinical/models')\
        .setInputCols(["document", 'token'])\
        .setOutputCol("embeddings")\
        .setCaseSensitive(False)

  chunk_emb = ChunkEmbeddings()\
        .setInputCols("chunk", "embeddings")\
        .setOutputCol("chunk_embeddings")

  icd10_resolver = ChunkEntityResolverModel.pretrained("chunkresolve_ICD10GM",'de','clinical/models') \
    .setInputCols(["token", "chunk_embeddings"]) \
    .setOutputCol("icd10_de_code")\
    .setDistanceFunction(distance)\
    .setNeighbours(10)
  #EUCLIDEAN, COSINE

  icd_pred_pipeline = Pipeline(
      stages = [
      document,
      chunk,
      tokenizer,
      stopwords,
      word_embeddings,
      chunk_emb,
      icd10_resolver])

  empty_df = spark.createDataFrame([['']]).toDF("text")

  icd_pred_model = icd_pred_pipeline.fit(empty_df)

  return icd_pred_model

In [ ]:
icd_pred_model = icd_phrase_resolver(distance='EUCLIDEAN') 
# distance >>'EUCLIDEAN' vs "COSINE"

stopwords_de download started this may take some time.
Approximate size to download 2.9 KB
[OK!]
w2v_cc_300d download started this may take some time.
Approximate size to download 1.2 GB
[OK!]
chunkresolve_ICD10GM download started this may take some time.
Approximate size to download 14.8 MB
[OK!]


In [ ]:
from pyspark.sql import functions as F

def get_icd10(text):

  text_df = spark.createDataFrame([[text]]).toDF("text")

  result_df = icd_pred_model.transform(text_df).select(F.explode(F.arrays_zip("chunk.result","chunk.metadata",
                                     "icd10_de_code.result","icd10_de_code.metadata")).alias("icd10_de_result")) \
    .select(F.expr("icd10_de_result['0']").alias("chunk"),
          F.expr("icd10_de_result['1'].entity").alias("entity"),
          F.expr("icd10_de_result['3'].resolved_text").alias("resolved_text"),
          F.expr("icd10_de_result['2']").alias("code"),
          F.expr("icd10_de_result['3'].all_k_resolutions").alias("alternative_resolved_texts"),
          F.expr("icd10_de_result['3'].all_k_results").alias("alternative_codes"))\
  .distinct() \
  .toPandas()
  
  return result_df

In [ ]:
text = 'Shigella dysenteriae'

res = get_icd10(text)

res

chunk  ...                                  alternative_codes
0  Shigella dysenteriae  ...  A03.0:::A04.73:::J15.7:::A04.70:::A04.6:::A04....

[1 rows x 6 columns]

In [ ]:
res['resolved_text'][0]

'Shigellose Shigella dysenteriae Shigellose Bakterielle Ruhr'

In [ ]:
res['alternative_resolved_texts'][0]

'Shigellose Shigella dysenteriae Shigellose Bakterielle Ruhr:::Enterokolitis Clostridium difficile Megakolon sonstigen Organkomplikationen Sonstige bakterielle Darminfektionen:::Pneumonie Mycoplasma pneumoniae Pneumonie Bakterien anderenorts klassifiziert:::Enterokolitis Clostridium difficile Megakolon sonstige Organkomplikationen Sonstige bakterielle Darminfektionen:::Enteritis Yersinia enterocolitica Sonstige bakterielle Darminfektionen:::Enterokolitis Clostridium difficile Megakolon sonstigen Organkomplikationen Sonstige bakterielle Darminfektionen:::Pneumonie Klebsiella pneumoniae Pneumonie Bakterien anderenorts klassifiziert:::Enterokolitis Clostridium difficile Sonstige bakterielle Darminfektionen:::Enterokolitis Clostridium difficile Megakolon sonstige Organkomplikationen Sonstige bakterielle Darminfektionen:::Meningitis Haemophilus influenzae Bakterielle Meningitis anderenorts klassifiziert'

In [ ]:
res['alternative_codes'][0]

'A03.0:::A04.73:::J15.7:::A04.70:::A04.6:::A04.71:::J15.0:::A04.7:::A04.72:::G00.0'

## Highlight Entities

In [ ]:
import random
from IPython.core.display import display, HTML


def get_color():
    r = lambda: random.randint(100,255)
    return '#%02X%02X%02X' % (r(),r(),r())

from spacy import displacy

def display_entities(annotated_text, filter_labels=True):

    label_list = []
    sent_dict_list = []
    
    for n in annotated_text['ner_chunk']:

        ent = {'start': n.begin, 'end':n.end+1, 'label':n.metadata['entity'].upper()}
        
        label_list.append(n.metadata['entity'].upper())

        sent_dict_list.append(ent)
   
    document_text = [{'text':annotated_text['document'][0].result, 'ents':sent_dict_list,'title':None}]

    label_list = list(set(label_list))
                
    label_color={}
    
    for l in label_list:
        
        label_color[l]=get_color()
    
    colors = {k:label_color[k] for k in label_list}
        
    
    html_text = displacy.render(document_text, style='ent', jupyter=True, manual=True, options= {"ents": label_list, 'colors': colors})

    return html_text

In [ ]:
text='''
Das Kleinzellige Bronchialkarzinom (Kleinzelliger Lungenkrebs, SCLC) ist Hernia femoralis, Akne, einseitig, ein hochmalignes bronchogenes Karzinom, das überwiegend im Zentrum der Lunge, in einem Hauptbronchus entsteht. Die mittlere Prävalenz wird auf 1/20.000 geschätzt. Vom SCLC sind hauptsächlich Peronen mittleren Alters (27-66 Jahre) mit Raucheranamnese betroffen. Etwa 70% der Patienten mit SCLC haben bei Stellung der Diagnose schon extra-thorakale Symptome. Zu den Symptomen gehören Thoraxschmerz, Dyspnoe, Husten und pfeifende Atmung. Die Beteiligung benachbarter Bereiche verursacht Heiserkeit, Dysphagie und Oberes Vena-cava-Syndrom (Obstruktion des Blutflusses durch die Vena cava superior). Zusätzliche Symptome als Folge einer Fernmetastasierung sind ebenfalls möglich. Rauchen und Strahlenexposition sind synergistisch wirkende Risikofaktoren. Die industrielle Exposition mit Bis (Chlormethyläther) ist ein weiterer Risikofaktor. Röntgenaufnahmen des Thorax sind nicht ausreichend empfindlich, um einen SCLC frühzeitig zu erkennen. Röntgenologischen Auffälligkeiten muß weiter nachgegangen werden, meist mit Computertomographie. Die Diagnose wird bioptisch gesichert. Patienten mit SCLC erhalten meist Bestrahlung und/oder Chemotherapie. In Hinblick auf eine Verbesserung der Überlebenschancen der Patienten ist sowohl bei ausgedehnten und bei begrenzten SCLC eine kombinierte Chemotherapie wirksamer als die Behandlung mit Einzelsubstanzen. Es kann auch eine prophylaktische Bestrahlung des Schädels erwogen werden, da innerhalb von 2-3 Jahren nach Behandlungsbeginn ein hohes Risiko für zentralnervöse Metastasen besteht. Das Kleinzellige Bronchialkarzinom ist der aggressivste Lungentumor: Die 5-Jahres-Überlebensrate beträgt 1-5%, der Median des gesamten Überlebens liegt bei etwa 6 bis 10 Monaten.
'''

ann_text = clinical_light_model.fullAnnotate(text)

display_entities (ann_text[0])